In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import tensorflow as tf

import sys
sys.path.append('/mnt/c/Users/kheut/code/covid19-forecasting/tf_model_1p5/')

from enum import Enum

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense

import tensorflow_probability as tfp
from scipy.stats import beta, truncnorm


# Local imports from model.py, data.py
from model import CovidModel, LogPoissonProb, get_logging_callbacks, Comp, Vax
from data import read_data, create_warmup
#from plots import make_all_plots

2021-10-14 05:09:57.080674: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-10-14 05:09:57.080717: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
transition_window =3

warmup_start = '20210428'
warmup_end = '20210430'
train_start = '20210501'
train_end = '20210731'
test_start = '20210801'
test_end = '20210831'

state = 'Massachusetts'
state_abbrev = 'MA'

data_dir = '../data'
covid_estim_date = '20210901'
hhs_date = '20210903'
owid_date = '20210903'

log_dir = './logs/new_warmup'

In [3]:
df = read_data(data_dir=data_dir,
               covid_estim_date=covid_estim_date,
               hhs_date=hhs_date,
               owid_date=owid_date,
               state=state, state_abbrev=state_abbrev)

In [4]:
class Comp(Enum):
    A = 0
    M = 1
    #X = 2
    #G = 3
    
class Vax(Enum):
    total = -1
    no = 0
    yes = 1

In [5]:
# get warmup arrays, splitting on vaccination status
warmup_asymp, warmup_mild, warmup_extreme = create_warmup(df, 
                                                          warmup_start, 
                                                          warmup_end,
                                                          0,0,0)

# re-combine
warmup_asymp = warmup_asymp[Vax.no.value] + warmup_asymp[Vax.yes.value]

In [6]:
delta = {}
delta[Vax.total.value] = {}
delta[Vax.total.value]['prior'] = {'a': 1.05, 'b': 20}
# delta is 0-1, so we will sigmoid the output of the variational posterior
# Here we use a reverse sigmoid to convert from a mean of 0.1
# I cant figure out the math to do for the scale parameter
delta[Vax.total.value]['posterior_init'] = {'loc': np.log(0.1/(1-0.1)), 'scale':0.1}

T_serial = {}
T_serial[Vax.total.value] = {}
T_serial[Vax.total.value]['prior'] ={'loc':5.8, 'scale':1}
# T_serial is positive, so we will softplus the output of the variational posterior
# Here we use a reverse softplus to convert from a mean of 5.8
# I cant figure out the math to do for the scale parameter
T_serial[Vax.total.value]['posterior_init'] = {'loc': tfp.math.softplus_inverse(5.8),
                                         'scale':0.1}


rho_M = {}
rho_M[Vax.total.value] = {}
rho_M[Vax.total.value]['prior'] = {'a': 31.8, 'b': 10.3}
# rho is 0-1, so we will sigmoid the output of the variational posterior
# Here we use a reverse sigmoid to convert from a mean of 0.76
# I cant figure out the math to do for the scale parameter
rho_M[Vax.total.value]['posterior_init'] = {'loc': np.log(0.76/(1-0.76)),
                                      'scale':0.1}

lambda_M = {}
lambda_M[Vax.total.value] = {}
lambda_M[Vax.total.value]['prior'] = {'loc': 4.7, 'scale': 1}
# lambda is positive, so we will softplus the output of the variational posterior
# Here we use a reverse softplus to convert from a mean of 4.7
# I cant figure out the math to do for the scale parameter
lambda_M[Vax.total.value]['posterior_init'] = {'loc': tfp.math.softplus_inverse(4.7),
                                         'scale':1}

nu_M = {}
nu_M[Vax.total.value] = {}
nu_M[Vax.total.value]['prior'] = {'loc': 3.1, 'scale': 1.2}
# nu is positive, so we will softplus the output of the variational posterior
# Here we use a reverse softplus to convert from a mean of 3.1
# I cant figure out the math to do for the scale parameter
nu_M[Vax.total.value]['posterior_init'] = {'loc': tfp.math.softplus_inverse(3.1),
                                     'scale':1.2}
delta = {}
delta[Vax.total.value] = {}
delta[Vax.total.value]['prior'] = {'a': 1.05, 'b': 20}
# delta is 0-1, so we will sigmoid the output of the variational posterior
# Here we use a reverse sigmoid to convert from a mean of 0.1
# I cant figure out the math to do for the scale parameter
delta[Vax.total.value]['posterior_init'] = {'loc': np.log(0.1/(1-0.1)), 'scale':0.1}

T_serial = {}
T_serial[Vax.total.value] = {}
T_serial[Vax.total.value]['prior'] ={'loc':5.8, 'scale':1}
# T_serial is positive, so we will softplus the output of the variational posterior
# Here we use a reverse softplus to convert from a mean of 5.8
# I cant figure out the math to do for the scale parameter
T_serial[Vax.total.value]['posterior_init'] = {'loc': tfp.math.softplus_inverse(5.8),
                                         'scale':0.1}


rho_M = {}
rho_M[Vax.total.value] = {}
rho_M[Vax.total.value]['prior'] = {'a': 31.8, 'b': 10.3}
# rho is 0-1, so we will sigmoid the output of the variational posterior
# Here we use a reverse sigmoid to convert from a mean of 0.76
# I cant figure out the math to do for the scale parameter
rho_M[Vax.total.value]['posterior_init'] = {'loc': np.log(0.76/(1-0.76)),
                                      'scale':0.1}

lambda_M = {}
lambda_M[Vax.total.value] = {}
lambda_M[Vax.total.value]['prior'] = {'loc': 4.7, 'scale': 1}
# lambda is positive, so we will softplus the output of the variational posterior
# Here we use a reverse softplus to convert from a mean of 4.7
# I cant figure out the math to do for the scale parameter
lambda_M[Vax.total.value]['posterior_init'] = {'loc': tfp.math.softplus_inverse(4.7),
                                         'scale':1}

nu_M = {}
nu_M[Vax.total.value] = {}
nu_M[Vax.total.value]['prior'] = {'loc': 3.1, 'scale': 1.2}
# nu is positive, so we will softplus the output of the variational posterior
# Here we use a reverse softplus to convert from a mean of 3.1
# I cant figure out the math to do for the scale parameter
nu_M[Vax.total.value]['posterior_init'] = {'loc': tfp.math.softplus_inverse(3.1),
                                     'scale':1.2}


2021-10-14 05:09:58.905260: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:923] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2d:00.0/numa_node
Your kernel may have been built without NUMA support.
2021-10-14 05:09:58.905467: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-10-14 05:09:58.905494: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2021-10-14 05:09:58.905516: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2021-10-14 05:09:58.905536: W tensorflow/stream_executor/platform/default/dso_loader.cc:6

In [7]:
warmup_A_params = {}
warmup_A_params[Vax.total.value] = {}
warmup_A_params[Vax.total.value]['prior'] = []
warmup_A_params[Vax.total.value]['posterior_init'] = []

for day in range(transition_window):
    warmup_A_params[Vax.total.value]['prior'].append({'loc': warmup_asymp[day],
                                                'scale': warmup_asymp[day]/10})
    # must be positive so reverse softplus the mean
    warmup_A_params[Vax.total.value]['posterior_init'].append({'loc': tfp.math.softplus_inverse(warmup_asymp[day]),
                                                         'scale': tfp.math.softplus_inverse(warmup_asymp[day]/10)})

In [8]:
model = CovidModel([Vax.total], [Comp.A, Comp.M],
                 transition_window,
                 delta, T_serial, rho_M, lambda_M, nu_M,
                 warmup_A_params, posterior_samples=1000)

In [15]:
model.call(df.loc[train_start:test_end,'Rt'].values)

IndexError: list index out of range

In [14]:
df.loc[train_start:test_end,'Rt'].values

array([0.9893859 , 0.9806603 , 0.9708389 , 0.96012908, 0.94874668,
       0.93690047, 0.92479089, 0.91261409, 0.90057085, 0.8888542 ,
       0.87764703, 0.86712298, 0.85742815, 0.84862583, 0.84075898,
       0.83387077, 0.82800523, 0.82319257, 0.81938499, 0.81651255,
       0.81450937, 0.81331291, 0.81286373, 0.81310665, 0.81398912,
       0.81545992, 0.81746881, 0.81997128, 0.82297143, 0.82650082,
       0.83059275, 0.83528213, 0.84060646, 0.84661472, 0.85336511,
       0.86091927, 0.86934258, 0.8787029 , 0.88903848, 0.90036034,
       0.9126806 , 0.92601353, 0.94037391, 0.95573397, 0.97201386,
       0.98912558, 1.00697454, 1.02545815, 1.04443745, 1.06372162,
       1.08310189, 1.10235411, 1.12123893, 1.13951188, 1.1569333 ,
       1.17325498, 1.18822025, 1.20156657, 1.21307565, 1.22264284,
       1.23018727, 1.23563722, 1.238931  , 1.2400631 , 1.2391775 ,
       1.23645443, 1.23208136, 1.22625046, 1.21916673, 1.21107649,
       1.20223337, 1.19288596, 1.1832766 , 1.17362909, 1.16410